<a href="https://colab.research.google.com/github/jordhanus/Reviews_POC/blob/main/Zero_shot_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's try different approaches here now, first zero-shot classification on other models.


In [70]:
# Libraries

# !pip install transformers
# !pip install datasets
# !pip install transformers[torch]
# !pip install evaluate
import pandas as pd

In [2]:
# Initialize the pipeline, see https://huggingface.co/facebook/bart-large-mnli

from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

# classifier2 = pipeline("zero-shot-classification",
#                       model="MoritzLaurer/deberta-v3-base-zeroshot-v1")

In [67]:
# Read data

df = pd.read_csv("Golden_dataset.csv")

In [68]:
# Function to process reviews based on categories, and assign scores based on zero-shot classification scores

categories = ['Customer experience', 'Delivery time', 'Quality of product']

def classify_review(review, categories=categories):

  expanded_categories_dict = {
    category: [f"Very poor {category}", f"Average {category}", f"Very good {category}", f"No information on {category}"]
    for category in categories
  }

  category_scores = {}

  # Classify and calculate score for each category
  for category in categories:
      result_classifier = classifier(review, expanded_categories_dict[category], multi_label=False)
      score_dict = dict(zip(result_classifier['labels'], result_classifier['scores']))
      # Calculate the custom score based on the classification results and your scoring logic
      if score_dict.get(f"No information on {category}",0) > 0.6:
        custom_score=0
      else:
        custom_score = score_dict.get(f'Average {category}', 0) * 2.7 + \
                        score_dict.get(f'Very good {category}', 0) * 4.5 + \
                        score_dict.get(f'Very poor {category}', 0) * 0.8
      # Store the custom score in the scores dictionary using the category name as key
      category_scores[f'Zero-shot model 1 {category}'] = custom_score

  # Convert the scores dictionary to a Series with category names as index and custom scores as values
  return pd.Series(category_scores)

# Apply the function to the proper column and assign the result to new columns in the dataframe
new_columns = df['review_text_unicode'].apply(classify_review)
df = pd.concat([df, new_columns], axis=1)

df.to_csv('Zero_shot_model1.csv')

In [69]:
df

,Unnamed: 0,name,review_text_unicode,review_rating,review_likes,review_year,review_date,owner_answer,owner_answer_date,answer_time,"('assessment', 'Customer experience')","('assessment', 'Delivery time')","('assessment', 'Quality of product')","('score', 'Customer experience')","('score', 'Delivery time')","('score', 'Quality of product')",Zero-shot model 1 Customer experience,Zero-shot model 1 Delivery time,Zero-shot model 1 Quality of product
0,0,Restaurant ABC,Absolutely love my new sofa! Staff was friendl...,1,10,2022,2022-01-27,Thank you for your feedback!,2022-01-28,1,"Friendly staff, prompt delivery",No issues with delivery,Love my new sofa,4,3,5,4.442331,4.404037,4.463916
1,1,Restaurant ABC,Bought a table that broke within a week. Staff...,3,47,2023,2023-11-24,We apologize for the inconvenience.,2023-11-27,3,Dismissive staff when called,No information,Table broke within a week,2,0,1,1.080071,1.057241,1.058607
2,2,Restaurant ABC,"Decent selection, but delivery took much longe...",0,9,2022,2022-10-26,Thank you for your feedback!,2022-11-07,12,Decent selection,Delivery took longer than promised,No information,3,2,0,2.065096,0.909236,2.710585
3,3,Restaurant ABC,"Quality furniture and great in-store service, ...",3,36,2021,2021-06-15,We apologize for the inconvenience.,2021-06-19,4,Great in-store service,Late delivery,Quality furniture,4,2,4,4.214893,1.077468,4.412679
4,4,Restaurant ABC,Staff were rude. I won't be coming back despit...,4,38,2022,2022-12-16,Thank you for your feedback!,2022-12-23,7,Rude staff,No information,Good product range,1,0,3,0.913299,1.701395,3.979776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Restaurant ABC,The table I received is of a different color. ...,5,36,2021,2021-12-21,We apologize for the inconvenience.,2021-12-23,2,"Table of different color, not as per image",No information,No information,2,0,0,1.943793,1.896119,1.978158
96,96,Restaurant ABC,Store has great bundle offers. Bought a comple...,5,19,2022,2022-03-03,Thank you for your feedback!,2022-03-12,9,"Great bundle offers, complete room set",No information,No information,4,0,0,4.072916,3.489498,4.348323
97,97,Restaurant ABC,The dresser I got is chipped. Not a pleasant e...,3,24,2022,2022-08-12,We apologize for the inconvenience.,2022-08-19,7,"Chipped dresser, not a pleasant experience",No information,No information,2,0,0,0.859753,0.992095,0.914293
98,98,Restaurant ABC,Stylish designs that are also functional. Very...,4,12,2023,2023-05-05,Thank you for your feedback!,2023-05-08,3,Stylish and functional designs,No information,No information,4,0,0,4.451081,3.816037,4.462884
